In [73]:
using JuMP, CPLEX, LinearAlgebra, DelimitedFiles, StatsPlots, Statistics, Measurements
plotly()
mc = readdlm("mcMeleeModel.csv", ',', Float64)
#mc = readdlm("mcSmashWiki2.csv",';',Float64)

mc = round.(mc, digits=2)

characters = ["Captain Falcon", "Donkey Kong", "Fox", "Mr. Game & Watch", "Kirby",
            "Bowser", "Link", "Luigi", "Mario", "Marth", "Mewtwo", "Ness", "Peach",
            "Pikachu", "Ice Climbers", "Jigglypuff", "Samus", "Yoshi", "Zelda",
            "Sheik", "Falco", "Young Link", "Dr. Mario", "Roy", "Pichu", "Ganondorf"]
;

In [84]:
function isViable(charater,playrate)
    
    model = Model(CPLEX.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable model x[1:26] >= 0
    @variable model  α
    @objective model Max α
    @constraint model sum(x) ==1
    
    for c in 1:26
        @constraint model α - x⋅mc[:,c] <=0 
    end
    
    @constraint model x[charater] == playrate
    optimize!(model)
    
    return termination_status(model) == MOI.OPTIMAL && isapprox(value(α),0.5,atol=0.005) 
    #value(α),value.(x)
end

isViable (generic function with 1 method)

In [85]:
function viableRange(charater,precision)
    
    r = isViable.(charater,0:precision:1)
    
    (findfirst(r)-1)*precision,(findlast(r)-1)*precision
end

viableRange (generic function with 1 method)

In [86]:
function plotRange(r)
    v = [mean(x) ± (x[2]-x[1])/2 for x in r]
   return scatter([1:length(r)...],v,markersize=0.1,thickness_scaling =2, yticks = 0:0.05:1,legend=false,linewidth =4, ylabel="Viable %")
end

plotRange (generic function with 1 method)

In [87]:
ranges = viableRange.(1:26,[0.005])

26-element Array{Tuple{Float64,Float64},1}:
 (0.0, 0.045)
 (0.0, 0.045)
 (0.0, 0.07)
 (0.0, 0.02)
 (0.0, 0.01)
 (0.0, 0.02)
 (0.0, 0.03)
 (0.0, 0.03)
 (0.0, 0.025)
 (0.0, 0.06)
 (0.0, 0.02)
 (0.0, 0.015)
 (0.88, 1.0)
 (0.0, 0.025)
 (0.0, 0.03)
 (0.0, 0.055)
 (0.0, 0.06)
 (0.0, 0.035)
 (0.0, 0.045)
 (0.0, 0.12)
 (0.0, 0.04)
 (0.0, 0.04)
 (0.0, 0.035)
 (0.0, 0.015)
 (0.0, 0.015)
 (0.0, 0.07)

In [88]:
plotRange(ranges)

<!DOCTYPE html>
 
 
 Plots.jl